# Keras

[Keras](https://keras.io/) 是一種較 Tensorflow 更高階的深度學習框架，建議先熟悉 [Tensorflow 單元](/notebooks/unit/tensorflow/tenforflow.ipynb)後再來閱讀本單元。Keras 由 Python 撰寫，可以使用少量的程式碼來建立深度學習模型。在使用時需要使用更低階的深度學習框架作為後端引擎，目前支援如 CNTK、Tensorflow、Theano 等知名框架。本單元將介紹 Keras 中 Model 與 Layer 的用法，並實作一個圖片分類器。本單元包含以下部分：

1. **Model** 介紹 Keras 的 Model。
2. **Layer** 介紹 Keras 的 Layer。
3. **CIFAR-10** 介紹圖片分類資料。
4. **範例模型**使用 Keras 實作兩種深度學習模型來進行圖片分類。

## 1. Model

###  [Sequential Model](https://keras.io/models/sequential/)

Sequential 是keras中一種常用的model型態，宣告完Sequential()之後可以使用add()函數來將Layer按照順序加入model中，被加入的layer就會依照加入的順序被執行出來。

## 2. Layer

###  [2.1 Dense Layer](https://keras.io/layers/core/#dense)

Dense Layer 就是普通型態的一層神經網路，假設X是輸入資料，則Dense Layer會學習一組參數W和常數b，使得經過這層的輸出等於 W*X + b。  

###  [2.2 Convolution Layer](https://keras.io/layers/convolutional/#conv2d)

Convolution Layer 是使用convolution來實作一層神經網路，有關於convolution的說明見[此網址](https://goo.gl/MDJbKt)。
  
在影像、圖片應用上，Convolution的表現比普通Dense Layer更為優異。  

### [2.3 MaxPooling](https://keras.io/layers/pooling/#maxpooling1d)

Pooling的機制是對眾多圖片的Pixel做篩選，篩選出分類器認為有用的局部特徵(feature)，篩選的機制有很多種，例如該Pixel在該範圍內的平均值、最大、最小值等等，在MaxPooling我們使用的是取局部最大值。

## 3. CIFAR-10

CIFAR 的全名為 Canadian Institute for Advanced Research，是由加拿大政府出資並由多位科學家、工程師收集而成的圖片資料庫。[CIFAR-10](http://www.cs.toronto.edu/~kriz/cifar.html) 包含 60000 張 32x32x3 的 RGB 彩色圖片，共 10 種分類。其中 50000 張為訓練資料，10000 張為測試資料。  

![CIFAR-10](./cifar_10.png)

Keras 提供[整理好的 CIFAR-10 資料](https://keras.io/datasets/#cifar10-small-image-classification)，只要透過 `import` 就可以拿到對應的訓練與測試資料。用法如下：

In [ ]:
%matplotlib inline

import matplotlib
import keras
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# show the i-th sample of the cifar-10 training set, try a different `i`
i = 0
matplotlib.pyplot.imshow(x_train[i])
print('Category:', y_train[i])

## 4. 範例模型

以下使用 Keras 實作兩種深度學習模型來進行 CIFAR-10 圖片分類。其中 DNN 只使用全連接層，而 CNN 多使用了卷積層。相較於[Tensorflow 單元](/notebooks/unit/tensorflow/tenforflow.ipynb) 的 MNIST 資料，CIFAR-10 的圖片比較複雜且為彩色，更能發揮卷積層的效果。也因此相較於 DNN，CNN 應該更容易得到好的結果。

In [ ]:
# DNN

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

x_tr_dnn = x_train[:10000].astype('float32')
x_te_dnn = x_test.astype('float32')

x_tr_dnn = x_tr_dnn.reshape(-1, 3072)
x_te_dnn = x_te_dnn.reshape(-1, 3072)

x_tr_dnn /= 255
x_te_dnn /= 255

y_tr_dnn = to_categorical(y_train[:10000], num_classes=10)
y_te_dnn = to_categorical(y_test, num_classes=10)

# options
epochs = 20
batch_size = 128 
learning_rate = 0.001

# model
model = Sequential()
model.add(Dense(100, activation='relu', input_shape=(3072,)))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# train
model.fit(x_tr_dnn, y_tr_dnn, batch_size=batch_size, epochs=epochs, shuffle=True, validation_data=(x_te_dnn, y_te_dnn))

In [ ]:
# CNN

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

x_tr_cnn = x_train[:10000].astype('float32')
x_te_cnn = x_test.astype('float32')

x_tr_cnn /= 255
x_te_cnn /= 255

y_tr_cnn = to_categorical(y_train[:10000], num_classes=10)
y_te_cnn = to_categorical(y_test, num_classes=10)

# options
epochs = 20
batch_size = 128 
learning_rate = 0.001

# model
model = Sequential()

# the input shape for cifar-10 is (32, 32, 3)
# use `Conv2D(#neurons, (filter_size))` to add convolutionary layers
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'))

# use `MaxPooling2D()` to add pooling layers
# model.add(MaxPooling2D((2, 2)))

# TODO: add more convolutionary and/or pooling layers here

# in practice, fully-connected layers are added after convolutionary and pooling ones 
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# train
model.fit(x_tr_cnn, y_tr_cnn, batch_size=batch_size, epochs=epochs, shuffle=True, validation_data=(x_te_cnn, y_te_cnn))

若依照以上範例的設定，經過 20 輪訓練後，DNN 可以到達約 40% 的正確率，而 CNN 可以達到約 50% 的正確率。請更改 DNN 或是 CNN 的架構來改善模型。提示：

1. 調整訓練輪數(`epochs`)
2. 調整批次大小(`batch_size`)
3. 調整學習速率(`learning_rate`)
4. 增加層數
5. 調整每層的神經元數量